In [1]:

include("src/botmarket.jl")
using .BotMarket

# session
world = ExternalWorld()

investors = [
    #ZeroIntelligentInvestor(Dict(generic_currency=>100_000_00, generic_stock=>2000); min=0, max=500000)
    ZeroIntelligentInvestor(Dict(generic_currency=>100_000_00, generic_stock=>1000_00); min=0, max=100_000_00)
    for i in 1:100
]
markets = [ContinuousDoubleAuctionMarket() for i in 1:1]
markets[1].last_price = 6000
trading_days = 500

include("main.jl")
run_simulation(investors, markets, trading_days)

------
1: Making order 6183 * 1388 BuyLimitOrder
------
3: Making order 6033 * 828 BuyLimitOrder
------
4: Making order 6120 * 75888 SellLimitOrder

------
Trade
Before buy 1388, sell 75888. Traders: 1 and 4
After buy 0, sell 74500
------

------
5: Making order 6218 * 12 BuyLimitOrder

------
Trade
Before buy 12, sell 74500. Traders: 5 and 4
After buy 0, sell 74488
------

------
7: Making order 6029 * 93956 SellLimitOrder

------
Trade
Before buy 828, sell 93956. Traders: 3 and 7
After buy 0, sell 93128
------

------
8: Making order 6097 * 848 BuyLimitOrder

------
Trade
Before buy 848, sell 93128. Traders: 8 and 7
After buy 0, sell 92280
------

------
9: Making order 6103 * 9479 SellLimitOrder
------
10: Making order 6079 * 52 BuyLimitOrder

------
Trade
Before buy 52, sell 92280. Traders: 10 and 7
After buy 0, sell 92228
------

------
12: Making order 6048 * 4442 SellLimitOrder
------
13: Making order 5940 * 19431 SellLimitOrder
------
14: Making order 6130 * 60191 SellLimitOrde

47: Making order 4213 * 52596 SellLimitOrder
------
48: Making order 4252 * 1798 BuyLimitOrder

------
Trade
Before buy 1798, sell 77579. Traders: 48 and 44
After buy 0, sell 75781
------

------
49: Making order 4197 * 1511 BuyLimitOrder

------
Trade
Before buy 1511, sell 75781. Traders: 49 and 44
After buy 0, sell 74270
------

------
52: Making order 4009 * 2225 SellLimitOrder

------
Trade
Before buy 2279, sell 2225. Traders: 31 and 52
After buy 54, sell 0
------

------
54: Making order 4129 * 87099 SellLimitOrder

------
Trade
Before buy 54, sell 87099. Traders: 31 and 54
After buy 0, sell 87045
------

------
55: Making order 4242 * 799 BuyLimitOrder

------
Trade
Before buy 799, sell 87045. Traders: 55 and 54
After buy 0, sell 86246
------

------
57: Making order 4193 * 8803 BuyLimitOrder

------
Trade
Before buy 8803, sell 86246. Traders: 57 and 54
After buy 0, sell 77443
------

------
58: Making order 4084 * 53858 SellLimitOrder
------
60: Making order 4050 * 318 BuyLimitO



------
Trade
Before buy 63, sell 84301. Traders: 76 and 88
After buy 0, sell 84238
------


------
Trade
Before buy 551, sell 84238. Traders: 80 and 88
After buy 0, sell 83687
------

------
89: Making order 2911 * 11611 SellLimitOrder
------
90: Making order 3208 * 631 BuyLimitOrder

------
Trade
Before buy 631, sell 11611. Traders: 90 and 89
After buy 0, sell 10980
------

------
91: Making order 2919 * 1722 BuyLimitOrder

------
Trade
Before buy 1722, sell 10980. Traders: 91 and 89
After buy 0, sell 9258
------

------
92: Making order 2960 * 24402 SellLimitOrder
------
93: Making order 2905 * 1608 BuyLimitOrder
------
94: Making order 2830 * 56360 SellLimitOrder

------
Trade
Before buy 1608, sell 56360. Traders: 93 and 94
After buy 0, sell 54752
------

------
96: Making order 2913 * 45919 SellLimitOrder
------
97: Making order 2881 * 22617 SellLimitOrder
------
1: Making order 2821 * 80360 SellLimitOrder
------
2: Making order 2970 * 586 BuyLimitOrder

------
Trade
Before buy 5

------
31: Making order 1366 * 70398 SellLimitOrder
------
36: Making order 1539 * 5 BuyLimitOrder

------
Trade
Before buy 5, sell 30288. Traders: 36 and 24
After buy 0, sell 30283
------

------
37: Making order 1424 * 1520 BuyLimitOrder

------
Trade
Before buy 1520, sell 30283. Traders: 37 and 24
After buy 0, sell 28763
------

------
38: Making order 1435 * 1699 BuyLimitOrder

------
Trade
Before buy 1699, sell 28763. Traders: 38 and 24
After buy 0, sell 27064
------

------
40: Making order 1432 * 64707 SellLimitOrder
------
43: Making order 1476 * 1606 BuyLimitOrder

------
Trade
Before buy 1606, sell 27064. Traders: 43 and 24
After buy 0, sell 25458
------

------
44: Making order 1263 * 58083 SellLimitOrder
------
45: Making order 1351 * 971 BuyLimitOrder

------
Trade
Before buy 971, sell 58083. Traders: 45 and 44
After buy 0, sell 57112
------

------
46: Making order 1361 * 25633 SellLimitOrder
------
47: Making order 1312 * 13969 BuyLimitOrder

------
Trade
Before buy 1396

------
58: Making order 331 * 58143 SellLimitOrder
------
59: Making order 502 * 95 BuyLimitOrder

------
Trade
Before buy 95, sell 41361. Traders: 59 and 57
After buy 0, sell 41266
------

Invalid order: 401 * 152 BuyLimitOrder (reserved 60952)
Unverserved: 13621, active amount 70701, reserved 0, positions: 13621
------
60: Making order 401 * 152 BuyLimitOrder
Invalid order: 401 * 152 BuyLimitOrder (asset ccy)


DomainError: DomainError with 60952:
Cannot reserve more asset (60952) than there is in the positions (Position: 13621, reserved: 0) 60

In [1]:

include("src/botmarket.jl")
using .BotMarket

# session
world = ExternalWorld()

investors = [
    #ZeroIntelligentInvestor(Dict(generic_currency=>100_000_00, generic_stock=>2000); min=0, max=500000)
    ZeroIntelligentInvestor(Dict(generic_currency=>100_000_00, generic_stock=>10_00); min=0, max=100_000_00)
    for i in 1:100
]
markets = [ContinuousDoubleAuctionMarket() for i in 1:1]
markets[1].last_price = 6000
trading_days = 1000

include("main.jl")
run_simulation(investors, markets, trading_days)

Reserving 722 of stock for 1 (0)
Reserving 2 of stock for 2 (0)
Reserving 286 of stock for 3 (0)
Reserving 241 of stock for 4 (0)
Reserving 665 of stock for 5 (0)
Reserving 766 of stock for 7 (0)
Reserving 8171320 of ccy for 8 (0)
Releasing 121960 of ccy for investor 8 (8171320)
Releasing 4366 of stock for investor 2 (2)


DomainError: DomainError with 4366:
Cannot release more asset (4366) than there is in the positions (1000)

## Junks

In [1]:
using Parameters
using Statistics
using Distributions
using Plots
using StatsPlots
using CSV
using DataFrames

# Standard library
using Random

In [2]:
include("src/botmarket.jl")
using .BotMarket

In [3]:
function save_positions(investors, path, assets=nothing)
    if isnothing(assets)
        assets = [
            :generic_currency, 
            :generic_stock,
        ]
    end

    df = DataFrame(
        name = [inv.name for inv in investors], 
    )
    for asset in assets
        df[:, asset] = [inv.positions[eval(asset)] for inv in investors]
    end
    CSV.write(path, df) # "results/pos_start.csv"
    df
end

save_positions (generic function with 2 methods)

In [4]:
# session
world = ExternalWorld()

investors = [
    ZeroIntelligentInvestor(Dict(generic_currency=>100_000_00, generic_stock=>2000); min=0, max=500000)
    for i in 1:10
]
markets = [ContinuousDoubleAuctionMarket() for i in 1:1]

trading_days = 500

500

In [ ]:
markets[1].last_price = 5000

In [5]:

save_positions(investors, "results/pos_start.csv")

trades = []
trd_day_of_trade = []
orderbook = []
for day in 1:trading_days
    # Trading day
    #println("Trading day ", day)
    trades_day = []
    for investor in investors, market in markets
        
        trds = place!(investor, market)
        
        if isnothing(trds)
            # Do nothing, cannot put to the same
            # check as below because the there
            # is no method isempty with nothing
        elseif ~isempty(trds)
            trades_day = vcat(trades_day, trds)
            trd_day_of_trade = vcat(trd_day_of_trade, repeat([day], length(trds)))
        end
    end
    trades = vcat(trades, trades_day)
    update!(world)
    #println("Price: $(markets[1].last_price), Trades: $(length(trades_day))")
    #for market in markets
    #    cancel_all!(market)
    #    
    #end
    #for inv in investors
        # Otherwise they cannot trade
    #    release_all!(inv)
    #end

end


save_positions(investors, "results/pos_end.csv")


df = DataFrame(
    timestamp = [trade.timestamp for trade in trades], 
    trading_day =  trd_day_of_trade,
    price = [trade.price for trade in trades],
    quantity = [trade.quantity for trade in trades],
    seller = [trade.seller.name for trade in trades],
    buyer = [trade.buyer.name for trade in trades],
)
CSV.write("results/trades.csv", df)

Reserving 1196 of stock for 1 (0)
Reserving 1896 of stock for 2 (0)
Reserving 5499989 of ccy for 3 (0)
Trading 11 with 3 (ccy) <--> 2 (stock) with price 499997 
Releasing 5499989 of ccy for investor 3 (5499989)
Releasing 11 of stock for investor 2 (1896)
Reserving 1498833 of ccy for 7 (0)
Reserving 4976890 of ccy for 9 (0)
Reserving 1814 of stock for 10 (0)
Trading 3 with 7 (ccy) <--> 10 (stock) with price 499611 
Releasing 1498833 of ccy for investor 7 (1498833)
Releasing 3 of stock for investor 10 (1814)
Trading 10 with 9 (ccy) <--> 10 (stock) with price 497689 
Releasing 4976890 of ccy for investor 9 (4976890)
Releasing 10 of stock for investor 10 (1811)
Releasing 0 of ccy for investor 3 (0)
Reserving 921 of stock for 3 (0)
Reserving 990494 of ccy for 4 (0)
Trading 2 with 4 (ccy) <--> 3 (stock) with price 494995 
Releasing 990494 of ccy for investor 4 (990494)
Releasing 2 of stock for investor 3 (921)
Reserving 191 of stock for 5 (0)
Reserving 6943188 of ccy for 6 (0)
Trading 14 wit

"results/trades.csv"

In [5]:

save_positions(investors, "results/pos_start.csv")

trades = []
trd_day_of_trade = []
orderbook = []
for day in 1:trading_days
    # Trading day
    #println("Trading day ", day)
    trades_day = []
    for investor in investors, market in markets
        
        trds = place!(investor, market)
        
        if isnothing(trds)
            # Do nothing, cannot put to the same
            # check as below because the there
            # is no method isempty with nothing
        elseif ~isempty(trds)
            trades_day = vcat(trades_day, trds)
            trd_day_of_trade = vcat(trd_day_of_trade, repeat([day], length(trds)))
        end
    end
    trades = vcat(trades, trades_day)
    update!(world)
    #println("Price: $(markets[1].last_price), Trades: $(length(trades_day))")
    #for market in markets
    #    cancel_all!(market)
    #    
    #end
    #for inv in investors
        # Otherwise they cannot trade
    #    release_all!(inv)
    #end

end


save_positions(investors, "results/pos_end.csv")


df = DataFrame(
    timestamp = [trade.timestamp for trade in trades], 
    trading_day =  trd_day_of_trade,
    price = [trade.price for trade in trades],
    quantity = [trade.quantity for trade in trades],
    seller = [trade.seller.name for trade in trades],
    buyer = [trade.buyer.name for trade in trades],
)
CSV.write("results/trades.csv", df)

Reserving 999998 of ccy for 1 (0)
Reserving 3499993 of ccy for 2 (0)
Reserving 1022 of stock for 4 (0)
Releasing 999998 of ccy for investor 1 (999998)
Releasing 2 of stock for investor 4 (1022)
Releasing 3499993 of ccy for investor 2 (3499993)
Releasing 7 of stock for investor 4 (1020)
Reserving 705 of stock for 5 (0)
Reserving 232 of stock for 6 (0)
Reserving 1144 of stock for 8 (0)
Releasing 0 of ccy for investor 1 (0)
Reserving 1717 of stock for 1 (0)
Releasing 0 of ccy for investor 2 (0)
Reserving 249 of stock for 2 (0)
Reserving 7986512 of ccy for 3 (0)
Releasing 7986512 of ccy for investor 3 (7986512)
Releasing 16 of stock for investor 1 (1717)
Releasing 1013 of stock for investor 4 (1013)
Reserving 964 of stock for 4 (0)
Releasing 705 of stock for investor 5 (705)
Reserving 867 of stock for 5 (0)
Releasing 232 of stock for investor 6 (232)
Reserving 1692 of stock for 6 (0)
Reserving 372 of stock for 7 (0)
Reserving 3470943 of ccy for 9 (0)
Releasing 3470943 of ccy for investor 9

DomainError: DomainError with 7837895072832:
Cannot release more asset (7837895072832) than there is in the positions (18433567)

In [9]:
using Pkg
Pkg.add("StatsBase")

  Updating registry at `C:\Users\Mxu\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.6 %0.0 % [===>                                     ]  6.2 %Fetching: [=====>                                   ]  12.4 % %Fetching: [==========>                              ]  24.7 %9 %38.9 %]  58.2 % %]  71.3 % [================================>        ]  77.8 %=====================================>   ]  92.4 %]  100.0 % Resolving package versions...
  Updating `C:\Users\Mxu\.julia\environments\v1.3\Project.toml`
  [2913bbd2] + StatsBase v0.32.2
  Updating `C:\Users\Mxu\.julia\environments\v1.3\Manifest.toml`
 [no changes]


In [45]:
using StatsBase
sample([1,2,3], Weights([0.25, 0.25, 0.5]))

2

In [2]:
get_order(investors[1], markets[1])

UndefVarError: UndefVarError: get_order not defined

In [3]:
"$(typeof([3,4]))"

"Array{Int64,1}"

In [7]:
mean([3,4])

3.5

In [13]:
rand(Uniform(sort([50,20])...))

36.06467904893677

In [4]:
rand(
            Uniform(
                sort([5, 5])...
            )
        )

ArgumentError: ArgumentError: Uniform: the condition a < b is not satisfied.

In [3]:
Uniform(2, 20)

Uniform{Float64}(a=2.0, b=20.0)